In [107]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from urllib.parse import urljoin

In [118]:
def scroll_and_click(url):

    """
    - 페이지 바닥까지 스크롤하면서 '기사 더보기' 버튼을 반복 클릭
    - 반복이 끝나면 현재 페이지의 모든 기사 링크를 수집하여 DataFrame 반환
    """
    
    global driver

    try:
        response = requests.get(url)
        response.raise_for_status()  # 문제가 발생하면 예외 발생
        html = response.text
    except requests.exceptions.RequestException as e:
        print(f"웹 페이지 요청 오류: {e}")
        exit()

    
    # XPath로 '기사 더보기' 버튼 찾기
    refresh_btn = driver.find_element(By.XPATH, '//a[contains(@class, "_CONTENT_LIST_LOAD_MORE_BUTTON")]')

    count = 0
    
    while True:
        count += 1
        print(f"[LOOP] {count} 번째 반복 중...")
        
        try:
            # 페이지 끝까지 스크롤하기
            body = driver.find_element(By.CSS_SELECTOR, 'body')
            for _ in range(10):  # 10번 정도 END 키 누르기
                body.send_keys(Keys.END)

            # 클릭
            # driver.execute_script("arguments[0].click();", refresh_btn)
            # time.sleep(1)  # 새 기사 로딩 대기

            # 최대 5초간 '기사 더보기' 버튼이 클릭 가능해질 때까지 기다림
            button = WebDriverWait(driver, 3).until(
                EC.element_to_be_clickable((By.XPATH, '//a[contains(@class, "_CONTENT_LIST_LOAD_MORE_BUTTON")]'))
            )
            button.click()
            print(f" → 버튼 클릭 성공 ({count}회)")
                        
        except Exception as e:
            print("기사 더보기 버튼 클릭 실패:", e)
            break

    """
    --- 불러온 HTMl에서 기사 링크 모두 추출 ---
    """

    # 버튼 클릭으로 새로 로드된 기사들은 Selenium이 로드한 DOM에만 존재하므로 request에는 안잡힘.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 뉴스 기사 링크 추출

    news_tag = soup.select('a.sa_text_title._NLOG_IMPRESSION')

    article_url_list = []

    for i in news_tag:
        href = i.get('href')
        full_url = urljoin(url, href)
        if href:
            article_url_list.append(full_url)

    df = pd.DataFrame({'url': article_url_list}).drop_duplicates(subset='url').reset_index(drop=True)
    return df


    

In [119]:
driver = None

def crawl_with_selenium(url):

    global driver

    # chrome path setting
    driver_path = "/workspace/chromedriver-linux64/chromedriver"
    chrome_path = "/workspace/chrome-linux64/chrome"
    
    chrome_options = webdriver.ChromeOptions()
    
    chrome_options.binary_location = chrome_path
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    
    service = Service(executable_path=driver_path)
    
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # __main__
    driver.get(url)

    article_url_df = scroll_and_click(url)# 정치-대통령실 사이트

    pd.set_option('display.max_colwidth', None)
    print(article_url_df.to_string(index=False))
    #print(article_url_df)  
    #print(driver.title)

    